# Retention Score Test
* StelllarAlgo Data Science
* Ryan Kazmerik & Nakisa Rad
* Mar 15, 2022

In [90]:
import getpass
import pyodbc

from datetime import datetime
from termcolor import colored, cprint

### Connect to SQL Server:

In [91]:
def get_sql_connection(dbName):
    
    SERVER = '34.206.73.189' 
    DATABASE = dbName
    USERNAME = 'dsAdminWrite' 
    PASSWORD = getpass.getpass(prompt='Enter your password')
    CNXN = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+SERVER+';DATABASE='+DATABASE+';UID='+USERNAME+';PWD='+ PASSWORD)
    
    return CNXN

### Create a dictionary of all teams that have retention scoring:

In [92]:
CNXN = get_sql_connection("datascience")
cursor = CNXN.cursor()

sql =  f"""
    SELECT DISTINCT
        clientcode,
        lkupclientid,
        stlrDBName,
        leagueName
    FROM 
        ds.productyear_all
    """

cursor = CNXN.cursor().execute(sql)
columns = [column[0] for column in cursor.description]

teams = []
for row in cursor.fetchall():
    teams.append(dict(zip(columns, row)))

print(f"Total teams: {len(teams)} ")
print(f"Example team {teams[0]} ")

Enter your password ··················


Total teams: 38 
Example team {'clientcode': '66ers', 'lkupclientid': 19, 'stlrDBName': 'stlrMILB', 'leagueName': 'MILB'} 


### Add two more fields to each teams dictionary (insertdate & count):

In [93]:
cursor = CNXN.cursor()

for team in teams:

    sql = f"""
            SELECT TOP (1) insertdate, count(*) as count
            FROM [DataScience].[DS].[customerScores]
            WHERE [lkupclientid] = {team['lkupclientid']}
            GROUP BY insertDate
            ORDER BY convert(datetime, insertdate) DESC
        """
    
    cursor = CNXN.cursor().execute(sql)

    results = []
    for row in cursor.fetchall():
        team['insertdate'] = row[0]
        team['count'] = row[1]

print(f"Example team {teams[0]} ")

Example team {'clientcode': '66ers', 'lkupclientid': 19, 'stlrDBName': 'stlrMILB', 'leagueName': 'MILB', 'insertdate': '03-15-2022 10:26:15', 'count': 255} 


### Write a test to see if insertdate is greater than todays date:

In [94]:
today = datetime.now().strftime("%m-%d-%Y 00:00:00")

print("SCORE REPORT FOR DATASCIENCE customerScores TABLE:")
for team in teams:
    
    if team['insertdate'] > today:
        print(colored(f" > {team['count']} Scores Updated: {team['clientcode']} ({team['lkupclientid']})", 'green'))
    else:
        print(colored(f" ! {team['count']} Scores NOT Updated: {team['clientcode']} ({team['lkupclientid']})", 'red'))

SCORE REPORT FOR DATASCIENCE customerScores TABLE:
 ! 255 Scores NOT Updated: 66ers (19)
 ! 1634 Scores NOT Updated: admirals (16)
 ! 5622 Scores NOT Updated: blazers (5)
 ! 3265 Scores NOT Updated: bulls (11)
 ! 5130 Scores NOT Updated: canucks (7)
 ! 2648 Scores NOT Updated: coyotes (55)
 ! 2296 Scores NOT Updated: drive (48)
 ! 5187 Scores NOT Updated: dynamo (13)
 ! 931 Scores NOT Updated: elpaso (30)
 ! 325 Scores NOT Updated: fireflies (28)
 ! 5381 Scores NOT Updated: flames (36)
 ! 7936 Scores NOT Updated: galaxy (6)
 > 105 Scores Updated: grizzlies (27)
 ! 429 Scores NOT Updated: hartfordyardgoats (44)
 ! 777 Scores NOT Updated: hitmen (37)
 ! 931 Scores NOT Updated: hops (9)
 ! 249 Scores NOT Updated: kanecounty (51)
 ! 16 Scores NOT Updated: legends (26)
 ! 603 Scores NOT Updated: loons (20)
 ! 3686 Scores NOT Updated: nhlpanthers (93)
 ! 1525 Scores NOT Updated: okcdodgers (47)
 ! 842 Scores NOT Updated: rainiers (17)
 ! 415 Scores NOT Updated: rattlers (24)
 ! 264 Scores NO

### Add another field to each team (dimdateid) from each teams STLR table:

In [80]:
for team in teams:
    
    if team['lkupclientid'] != 45: #exclude rays b/c they are in a diff db instance
    
        CNXN = get_sql_connection(team['stlrDBName'])
        cursor = CNXN.cursor()

        sql = f"""
                SELECT TOP (1) dimdateid
                FROM {team['stlrDBName']}.DW.factCustomerRetentionScore 
                WHERE [lkupclientid] = {team['lkupclientid']}
                GROUP BY dimdateid
                ORDER BY dimdateid DESC
            """

        cursor = CNXN.cursor().execute(sql)

        results = []
        for row in cursor.fetchall():
            team['dimdateid'] = row[0]
        
print(f"Example team {teams[0]} ")

Example team {'clientcode': '66ers', 'lkupclientid': 19, 'stlrDBName': 'stlrMILB', 'leagueName': 'MILB', 'insertdate': '03-15-2022 10:26:15', 'count': 255, 'dimdateid': 20220107} 


In [89]:
today = int(datetime.now().strftime("%Y%m%d"))

print("SCORE REPORT FOR STLR factCustomerRetentionScore TABLE:")
for team in teams:
    
    if team['lkupclientid'] != 45: #exclude rays b/c they are in a diff db instance

        if team['dimdateid'] >= today:
            print(colored(f" > {team['count']} Scores Updated: {team['clientcode']} ({team['lkupclientid']})", 'green'))
        else:
            print(colored(f" ! {team['count']} Scores NOT Updated: {team['clientcode']} ({team['lkupclientid']})", 'red'))

SCORE REPORT FOR STLR faceCustomerRetentionScore TABLE:
 ! 255 Scores NOT Updated: 66ers (19)
 ! 1634 Scores NOT Updated: admirals (16)
 > 5622 Scores Updated: blazers (5)
 ! 3265 Scores NOT Updated: bulls (11)
 > 5130 Scores Updated: canucks (7)
 > 2648 Scores Updated: coyotes (55)
 ! 2296 Scores NOT Updated: drive (48)
 > 5187 Scores Updated: dynamo (13)
 ! 931 Scores NOT Updated: elpaso (30)
 ! 325 Scores NOT Updated: fireflies (28)
 > 5381 Scores Updated: flames (36)
 > 7936 Scores Updated: galaxy (6)
 ! 105 Scores NOT Updated: grizzlies (27)
 ! 429 Scores NOT Updated: hartfordyardgoats (44)
 > 777 Scores Updated: hitmen (37)
 ! 931 Scores NOT Updated: hops (9)
 ! 249 Scores NOT Updated: kanecounty (51)
 ! 16 Scores NOT Updated: legends (26)
 ! 603 Scores NOT Updated: loons (20)
 ! 3686 Scores NOT Updated: nhlpanthers (93)
 ! 1525 Scores NOT Updated: okcdodgers (47)
 ! 842 Scores NOT Updated: rainiers (17)
 ! 415 Scores NOT Updated: rattlers (24)
 ! 1648 Scores NOT Updated: renoace